# BASE 0 - CI

Information:

* Database: base 0
* Predicted variable: Corrupción Intensa
* Predictor variables: Only SIAF and Canon
* Type of prediction: Clasification
* Period of training: 2016-2020

## 1. Cargar librerías, módulos y datos

In [1]:
import warnings
warnings.filterwarnings( 'ignore' )

In [2]:
import pandas as  pd
import numpy as np
import pickle
import joblib
from importlib.machinery import SourceFileLoader

In [3]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, f1_score, matthews_corrcoef
from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import KFold

In [ ]:
from econml.grf import RegressionForest
from lightgbm import LGBMClassifier

In [5]:
fun = SourceFileLoader( 'funciones', r'..\..\..\code\modules\funciones.py' ).load_module()
vn  = SourceFileLoader( 'variables_nombres', r'..\..\..\code\modules\variables_nombres.py' ).load_module()

In [7]:
path = r'..\..\..\input\preprocessed_data\base0_siaf_canon_1620.csv'
data = pd.read_csv( path )

## 2. Realizar la partición en conjunto de entrenamiento y prueba

In [17]:
dep_var    = [ 'corrup_intensa' ]
other_vars = [ 'monto_examinado', 'monto_auditado', 'monto_objeto_servicio',
               'monto_corrup1', 'monto_corrup2', 'tipo_control', 'corrup_amplia',
               'per_corrup1', 'per_corrup2', '_monto', 'monto_', 'year', 'ubigeo' ]

pred_vars  = [ col for col in data.columns if col not in dep_var and col not in other_vars ]

x_train, x_test, y_train, y_test = train_test_split( data[ pred_vars ],
                                                     data[ 'corrup_intensa' ],
                                                     test_size    = 0.3,
                                                     random_state = 2023,
                                                     stratify = data[ 'corrup_intensa' ] )

In [ ]:
# x_train_columns = x_train.columns.to_list()
# x_train_columns = pd.DataFrame( x_train_columns, columns = [ 'colname' ] )
# x_train_columns.to_excel( r'..\..\..\code\prediction\ejecucion_7\colnames_b0_ci_siaf.xlsx' )

## 3. Implementar métodos de muestreo

Se implementan los métodos SMOTE, SMOTE Tomek-Links y Naive Random Oversampling

In [18]:
x_train_s, x_train_st, x_train_nro, y_train_s, y_train_st, y_train_nro = fun.resampling( x_train, y_train )

In [19]:
x_train.name = 'x_train_o'
x_train_s.name = 'x_train_s'
x_train_st.name = 'x_train_st'
x_train_nro.name = 'x_train_nro'

In [20]:
check_data      = np.zeros( ( 4, 4 ) )

check_data[ 0 ] = [ x_train.shape[ 0 ], x_train.shape[ 1 ],
                    y_train.value_counts()[ 0 ], y_train.value_counts()[ 1 ] ]

check_data[ 1 ] = [ x_train_s.shape[ 0 ], x_train_s.shape[ 1 ],
                    y_train_s.value_counts()[ 0 ], y_train_s.value_counts()[ 1 ] ]

check_data[ 2 ] = [ x_train_st.shape[ 0 ], x_train_st.shape[ 1 ],
                    y_train_st.value_counts()[ 0 ], y_train_st.value_counts()[ 1 ] ]

check_data[ 3 ] = [ x_train_nro.shape[ 0 ], x_train_nro.shape[ 1 ],
                    y_train_nro.value_counts()[ 0 ], y_train_nro.value_counts()[ 1 ] ]

colnames        = [ 'observaciones', 'variables', 'Nro. No', 'Nro. Si' ]

rownames        = [ 'Original',
                    'SMOTE',
                    'SOMTE Tomek',
                    'NRS' ]

table_check_data = pd.DataFrame( check_data, columns = colnames )
table_check_data.index = rownames
table_check_data

,observaciones,variables,Nro. No,Nro. Si
Original,964.0,14317.0,272.0,692.0
SMOTE,1384.0,14317.0,692.0,692.0
SOMTE Tomek,1368.0,14317.0,684.0,684.0
NRS,1384.0,14317.0,692.0,692.0


## 4. Implementar los modelos de clasificación

In [22]:
o_20 = 20*x_train.shape[ 1 ]/100
o_30 = 30*x_train.shape[ 1 ]/100
o_40 = 40*x_train.shape[ 1 ]/100

print( o_20, o_30, o_40, sep = '\n' )

2863.4
4295.1
5726.8


In [ ]:
models = {

    'Logistic Regression'      : { 'model'      : LogisticRegression( random_state = 2023, n_jobs = -1 ) },

    'Logistic Lasso'           : { 'model'      : LogisticRegressionCV( random_state = 2023, penalty = 'l1', solver = 'saga', n_jobs = -1, Cs = [ 0.001, 0.01, 0.1, 1, 10, 100 ] ) },

    'Logistic Ridge'           : { 'model'      : LogisticRegressionCV( random_state = 2023, penalty = 'l2', solver = 'saga', n_jobs = -1, Cs = [ 0.001, 0.01, 0.1, 1, 10, 100 ] ) },

    'Logistic Elastic Net'     : { 'model'      : LogisticRegressionCV( random_state = 2023, penalty = 'elasticnet', solver = 'saga', l1_ratios = [ 0.5 ], n_jobs = -1, Cs = [ 0.001, 0.01, 0.1, 1, 10, 100 ] ) },

    'Random Forest Classifier' : { 'model'      : RandomForestClassifier( random_state = 2023, n_jobs = -1 ),
                                   'grid_params': { 'n_estimators': [ 250, 500, 1000 ], 'max_depth': [ 10, 20, 30 ], 'max_features': [ 2864, 4295, 5727 ] } },

    'XGboost Classifier'       : { 'model'      : XGBClassifier( random_state = 2023, use_label_encoder = False, objective = 'binary:logistic', verbosity = 0, learning_rate = 0.1, n_jobs = -1 ),
                                   'grid_params': { 'n_estimators': [ 250, 500, 1000 ], 'max_depth': [ 1, 2 ], 'max_features': [ 2864, 4295, 5727 ] } },


    'LGBMClassifier'           : { 'model'      : LGBMClassifier( random_state = 2023, n_jobs = -1 ),
                                   'grid_params': { 'n_estimators': [ 250, 500, 1000 ], 'max_depth': [ 1, 2 ] } },


        }

models_regression_forest = {
    'Regression Forest' : { 'model'      : RegressionForest( random_state = 2023, n_jobs = -1 ),
                            'grid_params': { 'n_estimators': [ 252, 500, 1000 ], 'max_depth': [ 10, 20, 30 ] } }
        }

In [ ]:
models_path  = r'..\..\..\output\ejecucion_8\models'
results_path = r'..\..\..\output\ejecucion_8\results'
vars_path    = r'..\..\..\output\ejecucion_8\vars'
gs_path      = r'..\..\..\output\ejecucion_8\gridsearch_results'

In [ ]:
x_train_list = [ x_train, x_train_s, x_train_st, x_train_nro ]
y_train_list = [ y_train, y_train_s, y_train_st, y_train_nro ]
path_list    = [ models_path, results_path, vars_path, gs_path ]

In [ ]:
sufix   = 'ci'

In [ ]:
# Main models

resultados = fun.test_models_classification( models, x_train_list, y_train_list, x_test, y_test, path_list, sufix )
resultados.to_excel( r'..\..\..\output\ejecucion_8\results\base0_ci.xlsx' )

In [ ]:
# Regression Forest Model

resultados_rf = fun.test_regression_forest( models_regression_forest, x_train_list, y_train_list, x_test, y_test, path_list, sufix )
resultados_rf.to_excel( r'..\..\..\output\ejecucion_8\results\base0_ci_regression_forest.xlsx' )